In [ ]:
import os
import lxml
import pytz
from lxml import etree
import datetime
from collections import defaultdict
import green_button
import solar_data

tz = pytz.timezone('US/Pacific')
mindate = datetime.date(2020, 6, 1)
maxdate = datetime.date(2020, 11, 6)
include_solar = True
include_meter = True


In [ ]:
import datetime
import numpy as np
import ipywidgets as ipw
import holoviews as hv
from holoviews import opts
    
hv.extension('bokeh')


def run(energy_contents, solar_contents, mindate, maxdate, height, width):
    datadict, dates = green_button.load_sce_data_from_xml(energy_contents, tz)
    if mindate is None:
        mindate = min(dates)
    if maxdate is None:
        maxdate = max(dates)
    solardatadict = solar_data.load_solar_data_from_json(solar_contents, tz)
    ndates = (maxdate - mindate).days + 1
    dates = [mindate + datetime.timedelta(days=i) for i in range(ndates)]
    tstimes = [f'{i//4:02}:{15*(i%4):02}' for i in range(96)]
    vals = np.zeros(shape=(96, ndates))

    if include_meter:
        for k, v in datadict.items():
            tsdate, tstime = k
            dateind = (tsdate - mindate).days
            if 0 <= dateind < ndates: 
                vals[tstime, dateind] += v

    if include_solar:
        for k, v in solardatadict.items():
            tsdate, tstime = k
            dateind = (tsdate - mindate).days
            if 0 <= dateind < ndates:
                vals[tstime, dateind] += v
                
    heatmap = hv.HeatMap([(datetime.datetime(dates[j].year, dates[j].month, dates[j].day),tstimes[i],vals[i,j]) for i in range(96) for j in range(ndates)], kdims=['date', 'time'], vdims=['energy']).options(yticks=range(0,96,4))
    per_day = hv.Curve([(datetime.datetime(dates[j].year, dates[j].month, dates[j].day), vals[:,j].sum()) for j in range(ndates)], kdims=['date'], vdims=['energy'])
    
    display(hv.Layout([heatmap,per_day]).cols(1).opts([opts.HeatMap(tools=['hover'], colorbar=True, height=height, width=width, toolbar='above'),
                                   opts.Curve(tools=['hover'], width=width)]))
    return mindate, maxdate

def plotly_version():
    import plotly.graph_objects as go
    fig = go.Figure(data=go.Heatmap(
            z=vals,
            x=dates,
            y=tstimes,
            #colorscale='Viridis'
        )
    )

    fig.update_layout(
        title='Power usage',
        xaxis_nticks=36,
        #autosize=False,
        #width=5000,
        #height=2000,
        #margin=dict(
        #    l=50,
        #    r=50,
        #    b=100,
        #    t=100,
        #    pad=4
        #),
    )

    fig.show()


In [ ]:
from IPython.display import display, clear_output
startipw = ipw.DatePicker()
endipw = ipw.DatePicker()
runbtn = ipw.Button(description="run", layout=ipw.Layout(width="45px"))
energy_uploader = ipw.FileUpload(accept='.xml', multiple=True)
solar_uploader = ipw.FileUpload(accept='.json', multiple=True)

iheight = ipw.IntText(value=500)
iwidth = ipw.IntText(value=1200)

label1 = ipw.Label("Upload SCE usage data xml here: ")
label2 = ipw.Label("Upload solar production data json here: ")
label3 = ipw.Label("Wdith and height: ")

def on_click(btn):
    clear_output()
    display(show_widget())
    s = startipw.value
    e = endipw.value
    energy_contents = [v['content'] for v in energy_uploader.value.values()]
    solar_contents = [v['content'] for v in solar_uploader.value.values()]
    mindate, maxdate = run(energy_contents, solar_contents, s, e, iheight.value, iwidth.value)
    startipw.value = mindate
    endipw.value = maxdate
        
runbtn.on_click(on_click)

def show_widget():
    return ipw.VBox((ipw.HBox((label1, energy_uploader)), ipw.HBox((label2, solar_uploader)), startipw, endipw,
                     ipw.HBox((label3, iwidth, iheight)), runbtn))
    
show_widget()